# Lab 2

In [155]:
%pip install plotly
%pip install mlxtend


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning:

This process (pid=21866) is multi-threaded, use of forkpty() may lead to deadlocks in the child.




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning:

This process (pid=21866) is multi-threaded, use of forkpty() may lead to deadlocks in the child.




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [73]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from mlxtend.classifier import OneRClassifier

## Utility Functions

In [5]:
def draw_plot(X, y, title=""):
    """
    Draw a scatter plot of the data.
    """
    
    fig = go.Figure(data=go.Scatter(x=X[:, 0], y=X[:, 1], mode='markers', marker=dict(color=y)))
    fig.update_layout(
        title=title,
        xaxis_title="X1",
        yaxis_title="X2",
        showlegend=False,
        width=600,
        height=600,
    )
    fig.show()

In [6]:
def train_ml_model(model_type, X, y, test_size=0.2, random_state=1505, **model_params):
    """
    Train a machine learning model based on the specified type.
    
    Parameters:
    - model_type (str): Type of model to train.
    - X (array-like): Feature matrix
    - y (array-like): Target variable
    - test_size (float): Proportion of the dataset to include in the test split
    - random_state (int): Random seed for reproducibility
    - **model_params: Additional parameters to pass to the model
    
    Returns:
    - X_train, X_test, y_train, y_test: Split datasets
    - model: Trained model
    """

    # Setting seed
    np.random.seed(random_state)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )
    
    # Initialize the model based on the model_type
    model = None
    if model_type == 'naive_bayes':
        model = GaussianNB(**model_params)
    elif model_type == 'logistic':
        model = LogisticRegression(**model_params)
    elif model_type == 'decision_tree':
        model = DecisionTreeClassifier(**model_params)
    elif model_type == 'svm':
        model = SVC(probability=True, **model_params)
    elif model_type == 'nearest_neighbors':
        model = KNeighborsClassifier(**model_params)
    elif model_type == 'decision_rules':
        model = OneRClassifier(**model_params)
    else:
        raise ValueError(f"Unknown model type: {model_type}. Available options: 'logistic', 'decision_tree', 'random_forest', 'svm'")
    
    # Train the model
    model.fit(X_train, y_train)
    
    return X_train, X_test, y_train, y_test, model

In [80]:
def draw_naive_bayes_plot(X_train, X_test, Y_test, nb_model, fig, row, col):
    
    # Defining discrete colors for the 4 classes
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Blue, Orange, Green, Red

    # Create decision boundary visualization
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1

    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 100),
        np.linspace(y_min, y_max, 100)
    )

    # Create predictions for the mesh grid
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    Z = nb_model.predict(mesh_points)
    Z = Z.reshape(xx.shape)

    # Decision boundaries
    fig.add_trace(
        go.Contour(
            x=np.linspace(x_min, x_max, 100),
            y=np.linspace(y_min, y_max, 100),
            z=Z,
            colorscale='viridis',
            opacity=0.3,
            showscale=False,
            hoverinfo='skip'
        ),
        row=row, col=col
    )

    # Adding the test points
    fig.add_trace(
        go.Scatter(
            x=X_test[:, 0],
            y=X_test[:, 1],
            mode='markers',
            marker=dict(
                color=[colors[i] for i in Y_test],
                size=5,
                line=dict(width=0.5, color='black')
            ),
            name='Test Data',
            showlegend=False
        ),
        row=row, col=col
    )

In [60]:
def draw_decision_tree_plot(X_train, X_test, Y_test, dt_model, fig, row, col):
    
    # Defining discrete colors for the 4 classes
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']  # Blue, Orange, Green, Red
    
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1

    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                        np.linspace(y_min, y_max, 300))
    Z = dt_model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Add decision boundaries
    fig.add_trace(go.Contour(
        x=np.linspace(x_min, x_max, 300),
        y=np.linspace(y_min, y_max, 300),
        z=Z,
        colorscale='Viridis',
        opacity=0.4,
        showscale=False,
        contours=dict(showlines=True, coloring='fill')
    ), row=row,col=col)

    # Add only test points
    for cls in np.unique(Y_test):
        fig.add_trace(go.Scatter(
            x=X_test[Y_test == cls, 0],
            y=X_test[Y_test == cls, 1],
            mode='markers',
            name=f'Class {cls}',
            marker=dict(
                color=colors[cls],
                size=5, line=dict(width=0.5, color='black')
            )
        ), row=row, col=col)

In [74]:
def _to_df(X: np.ndarray, y: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame({"x1": X[:, 0], "x2": X[:, 1], "y": y.astype(int)})

## Task 1: Pairing dataset

In [7]:
pairs = [
    ("naive_bayes", "decision_tree"),
    ("logistic_regression", "random_forest"),
    ("svm", "nearest_neighbors")
]

## Task 2: Data generation and visualization

### Synthetic Data for Naive Bayes

In [8]:
def synthetic_dataset_nb(n_samples: int = 500, noise_scale: float = 0.45, random_state: int = 1505):
    """
    Generate a synthetic dataset for Naive Bayes.
    Description:
    - Linearly separable classes
    - No redundant features
    - No correlation between features
    """

    np.random.seed(random_state)
    
    # Create diagonal centers
    centers = np.array([
        [2, 2],   # Class 0
        [4, 4],   # Class 1
        [6, 6],   # Class 2
        [8, 8]    # Class 3
    ])

    n_samples_per_class = int(n_samples // len(centers))

    X_list = []
    y_list = []

    for i, (cx, cy) in enumerate(centers):
        x1 = np.random.normal(cx, scale = noise_scale, size = n_samples_per_class)
        x2 = np.random.normal(cy, scale = noise_scale, size = n_samples_per_class)
        X_list.append(np.vstack([x1, x2]).T)
        y_list.append(np.full(n_samples_per_class, i))

    X = np.vstack(X_list)
    y = np.concatenate(y_list)

    return X, y
    

### Synthetic Data for Logistic Regression

In [ ]:
def synthetic_dataset_lr(n_samples : int = 500, correlation : float = 0.3, noise_scale : float = 0.5, random_state : int = 1505):
    """Generate a synthetic dataset for Logistic Regression.
    Description: 
    - No feature independence assumption
    - Linear separability
    """
    np.random.seed(random_state)
    
    # Create class means (arranged in a square)
    class_means = np.array([
        [2, 2],     # Class 0: top-right
        [-2, 2],    # Class 1: top-left
        [-2, -2],   # Class 2: bottom-left
        [2, -2]     # Class 3: bottom-right
    ])

    # Create data for each class
    X_list = []
    y_list = []

    n_samples_per_class = int(n_samples // len(class_means))

    for class_idx, mean in enumerate(class_means):
        # Generate points around each class mean with small variance
        X_class = np.random.normal(loc=mean, scale=0.5, size=(n_samples_per_class, 2))
        # Add some noise to make the classes less separable
        X_class += np.random.normal(scale=noise_scale, size=X_class.shape)
        X_list.append(X_class)
        y_list.append(np.full(n_samples_per_class, class_idx))

    # Combine all classes
    X = np.vstack(X_list)
    y = np.concatenate(y_list)

    # Create covariance matrix
    cov_matrix = np.array([
        [1.0, correlation],  # Variance of X1 = 1, Cov(X1,X2) = rho
        [correlation, 1.0]   # Cov(X1,X2) = rho, Variance of X2 = 1
    ])

    # Apply Cholesky decomposition to get transformation matrix
    L = np.linalg.cholesky(cov_matrix)

    # Apply the transformation to add correlation
    X_correlated = X @ L.T

    return X_correlated, y

### Synthetic Data for Support Vector Machine (Linear Kernel)

In [92]:
def synthetic_dataset_svm(n_samples: int = 500, correlation: float = 0.5, noise_scale: float = 0.4, random_state: int = 1505):
    """Generate a synthetic dataset for Support Vector Machine (Linear Kernel).
    Description: 
    - No feature independence assumption
    - Linear separability
    - Doesnt handle well overlapping classes. So let's define less noise scale.
    """
    np.random.seed(random_state)    

    # Create class means (arranged in a square)
    class_means = np.array([
        [2, 2],     # Class 0: top-right
        [-2, 2],    # Class 1: top-left
        [-2, -2],   # Class 2: bottom-left
        [2, -2]     # Class 3: bottom-right
    ])

    n_samples_per_class = int(n_samples // len(class_means))

    # Create data for each class
    X_list = []
    y_list = []

    for class_idx, mean in enumerate(class_means):
        # Generate points around each class mean with small variance
        X_class = np.random.normal(loc=mean, scale=0.5, size=(n_samples_per_class, 2))
        # Add some noise to make the classes less separable
        X_class += np.random.normal(scale=noise_scale, size=X_class.shape)
        X_list.append(X_class)
        y_list.append(np.full(n_samples_per_class, class_idx))

    # Combine all classes
    X = np.vstack(X_list)
    y = np.concatenate(y_list)

    # Create covariance matrix
    cov_matrix = np.array([
        [1.0, correlation],  # Variance of X1 = 1, Cov(X1,X2) = rho
        [correlation, 1.0]   # Cov(X1,X2) = rho, Variance of X2 = 1
    ])

    # Apply Cholesky decomposition to get transformation matrix
    L = np.linalg.cholesky(cov_matrix)

    # Apply the transformation to add correlation
    X_correlated = X @ L.T

    return X_correlated, y

### Synthetic Data for Decision Trees

In [76]:
def synthetic_dataset_dt(n_samples: int = 600, noise_scale: float = 0.2, correlation: float = 0.5, random_state: int = 1505) -> tuple[np.ndarray, np.ndarray]:
    """
    Generate a synthetic dataset for Decision Trees.
    Description: 
    XOR structure: for correlation.
    NB independence assumption fails; Tree captures interaction well.
    """
    rng = np.random.default_rng(random_state)

    # Sample uniformly from a square; labels assigned via unions of rectangles.
    X = rng.uniform(-1.0, 1.0, size=(n_samples, 2))
    x1, x2 = X[:, 0], X[:, 1]

    # Define axis-aligned regions (unions) -> simple for trees, nasty for NB
    # Class 0: two opposite corner boxes (same-sign corners)
    c0 = ((x1 < -0.5) & (x2 < -0.5)) | ((x1 > 0.5) & (x2 > 0.5))

    # Class 1: the other two opposite corners (opposite-sign corners)
    c1 = ((x1 < -0.5) & (x2 > 0.5)) | ((x1 > 0.5) & (x2 < -0.5))

    # Class 2: horizontal bands around x2=0 but away from vertical center in x1
    c2 = ((np.abs(x2) < 0.18) & (np.abs(x1) > 0.35))

    # Class 3: vertical bands around x1=0 but away from horizontal center in x2
    c3 = ((np.abs(x1) < 0.18) & (np.abs(x2) > 0.35))

    # Some points might fall into no region or overlaps; prioritize corners > bands.
    # Initialize with -1 (unassigned), then assign in order of priority.
    y = np.full(n_samples, -1, dtype=int)
    y[c0] = 0
    y[(y < 0) & c1] = 1
    y[(y < 0) & c2] = 2
    y[(y < 0) & c3] = 3

    # Any remaining unassigned points -> sprinkle to nearest rule by tie-breaker
    # (keeps classes balanced-ish and regions crisp for trees)
    unassigned = (y < 0)
    if np.any(unassigned):
        xa, ya = x1[unassigned], x2[unassigned]
        # Simple nearest-region heuristic based on distances to region centers:
        # centers roughly match the big areas above
        proto_centers = np.array([
            [-0.75, -0.75],  # class 0 region A
            [ 0.75,  0.75],  # class 0 region B
            [-0.75,  0.75],  # class 1 region A
            [ 0.75, -0.75],  # class 1 region B
            [-0.60,  0.00],  # class 2 left band
            [ 0.60,  0.00],  # class 2 right band
            [ 0.00, -0.60],  # class 3 lower band
            [ 0.00,  0.60],  # class 3 upper band
        ])
        pts = np.stack([xa, ya], axis=1)[:, None, :]
        dists = np.sum((pts - proto_centers[None, :, :])**2, axis=2)
        nearest = np.argmin(dists, axis=1)
        # Map proto center index to class
        idx2cls = np.array([0, 0, 1, 1, 2, 2, 3, 3], dtype=int)
        y[unassigned] = idx2cls[nearest]

    
    return X, y
    # np.random.seed(random_state)

    # n_q = n_samples // 4
    # class_means = [(-1.5, -1.5), (1.5, 1.5), (-1.5, 1.5), (1.5, -1.5)]
    # data = []
    # labels = []
    # for i, (cx, cy) in enumerate(class_means):
    #     pts = np.random.normal(loc=[cx, cy], scale=noise_scale, size=(n_q, 2))
    #     data.append(pts)
    #     labels.append(np.full(n_q, i))
    # X = np.vstack(data)
    # y = np.hstack(labels)

    # # Create covariance matrix
    # cov_matrix = np.array([
    #     [1.0, correlation],  # Variance of X1 = 1, Cov(X1,X2) = rho
    #     [correlation, 1.0]   # Cov(X1,X2) = rho, Variance of X2 = 1
    # ])

    # # Apply Cholesky decomposition to get transformation matrix
    # L = np.linalg.cholesky(cov_matrix)

    # # Apply the transformation to add correlation
    # X_correlated = X @ L.T

    # return X_correlated, y

### Synthetic Data for Decision Rules

In [136]:
def synthetic_dataset_dr(n_samples: int =600, noise_scale: float =0.05, random_state: int =1505) -> tuple[np.ndarray, np.ndarray]:
    """
    Generate a synthetic dataset for Decision Rules.
    Axis-aligned rectangular rules:
      - Class 0: x1 <= 0 and x2 <= 0
      - Class 1: x1 > 0 and x2 <= 0
      - Class 2: x1 <= 0 and x2 > 0
      - Class 3: x1 > 0 and x2 > 0
    Shallow trees (decision rules) capture rectangles cleanly.
    """
    np.random.seed(random_state)
    
    # Uniform points in 2D space
    X = np.random.uniform(low=-2.0, high=2.0, size=(n_samples, 2))

    # Assign class based on axis-aligned rules (quadrants)
    y = np.zeros(n_samples, dtype=int)
    y[(X[:,0] > 0) & (X[:,1] <= 0)] = 1
    y[(X[:,0] <= 0) & (X[:,1] > 0)] = 2
    y[(X[:,0] > 0) & (X[:,1] > 0)] = 3

    # Add noise: flip labels for a fraction of samples
    idx = np.random.choice(n_samples, size=int(noise_scale*n_samples), replace=False)
    y[idx] = np.random.randint(0, 4, size=len(idx))

    return X, y

### Synthetic Data for Nearest Neighbor

In [129]:
def synthetic_dataset_knn(n_samples: int = 600, noise_scale: float = 0.6, random_state: int = 42) -> tuple[np.ndarray, np.ndarray]:
    """
    Synthetic dataset designed to favor kNN over SVM.
    - kNN advantages:
    The same class is contiguous in feature space.
    There's no dependency between features.
    - SVM disadvantages:
    The boundaries are not linearly separable.
    """
    
    np.random.seed(random_state)
    X, y = [], []
    n_classes = 4
    
    # Define radii and calculate samples per class
    radii = [i for i in range(n_classes)]
    n_samples_per_class = int(n_samples // len(radii))
    
    # Generate data
    for i, r in enumerate(radii):
        theta = np.linspace(0, 2 * np.pi, n_samples_per_class, endpoint=False)
        circle = np.c_[r * np.cos(theta), r * np.sin(theta)]
        circle += np.random.normal(0, noise_scale, circle.shape)
        X.append(circle)
        y.extend([i] * n_samples_per_class)
    
    X = np.vstack(X)
    y = np.array(y)
    return X, y

### Naive Bayes

In [9]:
X,y = synthetic_dataset_nb(n_samples=500, noise_scale=0.5, random_state=1505)
print("Correlation:", np.corrcoef(X[:,0], X[:,1])[0,1])
# Plotting the data
draw_plot(X,y,"Synthetic dataset for Naive Bayes")


Correlation: 0.9523868543691388


### Decision Tree

### Logistic Regression

In [97]:
X,y = synthetic_dataset_lr(n_samples=500, correlation=-0.4, noise_scale=0.7)
correlation = np.corrcoef(X[:,0], X[:,1])[0,1]
print("Correlation:", correlation)
draw_plot(X,y,"Synthetic dataset for Logistic Regression")


Correlation: -0.4194113310085513


### Support Vector Machine

In [98]:
X, y = synthetic_dataset_svm(n_samples=500, correlation=0.8)
correlation = np.corrcoef(X[:,0], X[:,1])[0,1]
print("Correlation:", correlation)
draw_plot(X,y,"Synthetic dataset for SVM (Linear Kernel)")

Correlation: 0.8016553008473553


In [77]:
X, y = synthetic_dataset_dt(n_samples=500, random_state=1505)
correlation = np.corrcoef(X[:,0], X[:,1])[0,1]
print("Correlation:", correlation)
draw_plot(X,y,"Synthetic dataset for Decision Tree")

Correlation: 0.0059536776068985576


### Decision Rules

In [100]:
X, y = synthetic_dataset_dr(n_samples=500, noise_rate=0.01, random_state=1505)
correlation = np.corrcoef(X[:,0], X[:,1])[0,1]
print("Correlation:", correlation)
draw_plot(X,y,"Synthetic dataset for Decision Rules")

Correlation: 0.0028942143292361093


### Nearest Neighbor

In [101]:
X, y = synthetic_dataset_knn(n_samples=500, std=0.8, noise=0.2, random_state=42)
correlation = np.corrcoef(X[:,0], X[:,1])[0,1]
print("Correlation:", correlation)
draw_plot(X,y,"Synthetic dataset for Nearest Neighbor")

Correlation: -0.004685765220882318


## Task: 3 Train models

### Naive Bayes vs Logistic Regression

In [54]:
# Naive Bayes Dataset
X_nbb, Y_nbb = synthetic_dataset_nb(n_samples=500, noise_scale=0.8, random_state=1505)
X_nbb_nbb_train, X_nbb_nbb_test, Y_nbb_nbb_train, Y_nbb_nbb_test, model_nbb_nbb = train_ml_model(model_type='naive_bayes', X=X_nbb, y=Y_nbb, test_size=0.2, random_state=1505)
X_nbb_dt_train, X_nbb_dt_test, Y_nbb_dt_train, Y_nbb_dt_test, model_nbb_dt = train_ml_model(model_type='decision_tree', X=X_nbb, y=Y_nbb, test_size=0.2, random_state=1505)

# Accuracy
print(f"Naive Bayes Accuracy: {accuracy_score(Y_nbb_nbb_test, model_nbb_nbb.predict(X_nbb_nbb_test))}")
print(f"Decision Tree Accuracy: {accuracy_score(Y_nbb_dt_test, model_nbb_dt.predict(X_nbb_dt_test))}")


Naive Bayes Accuracy: 0.98
Decision Tree Accuracy: 0.95


In [61]:
# Create subplots
fig = go.Figure()
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Naive Bayes Model", " Decision Tree Model"),
    specs=[[{"type": "xy"}, {"type": "xy"}]]
)

draw_naive_bayes_plot(X_nbb_nbb_train, X_nbb_nbb_test, Y_nbb_nbb_test, model_nbb_nbb, fig, 1, 1)
draw_decision_tree_plot(X_nbb_nbb_train, X_nbb_nbb_test, Y_nbb_nbb_test, model_nbb_dt, fig, 1, 2)

# Update layout
fig.update_layout(
    title="Model Performance With Naive Bayes Friendly Data",
    width=800,
    height=400
)

fig.update_xaxes(title_text="X1")
fig.update_yaxes(title_text="X2")

fig.show()

In [78]:
# Decision Tree Dataset
X_dt, Y_dt = synthetic_dataset_dt(n_samples=500, noise_scale=0.7, random_state=1505)
X_dt_nbb_train, X_dt_nbb_test, Y_dt_nbb_train, Y_dt_nbb_test, model_dt_nbb = train_ml_model(model_type='naive_bayes', X=X_dt, y=Y_dt, test_size=0.2, random_state=1505)
X_dt_dt_train, X_dt_dt_test, Y_dt_dt_train, Y_dt_dt_test, model_dt_dt = train_ml_model(model_type='decision_tree', X=X_dt, y=Y_dt, test_size=0.2, random_state=1505)

# Accuracy
print(f"Naive Bayes Accuracy: {accuracy_score(Y_dt_nbb_test, model_dt_nbb.predict(X_dt_nbb_test))}")
print(f"Decision Tree Accuracy: {accuracy_score(Y_dt_dt_test, model_dt_dt.predict(X_dt_dt_test))}")


Naive Bayes Accuracy: 0.67
Decision Tree Accuracy: 0.95


In [81]:
# Create subplots
fig = go.Figure()
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Naive Bayes Model", " Decision Tree Model"),
    specs=[[{"type": "xy"}, {"type": "xy"}]]
)

draw_naive_bayes_plot(X_dt_nbb_train, X_dt_nbb_test, Y_dt_nbb_test, model_dt_nbb, fig, 1, 1)
draw_decision_tree_plot(X_dt_dt_train, X_dt_dt_test, Y_dt_dt_test, model_dt_dt, fig, 1, 2)

# Update layout
fig.update_layout(
    title="Model Performance With Naive Bayes Friendly Data",
    width=800,
    height=400
)

fig.update_xaxes(title_text="X1")
fig.update_yaxes(title_text="X2")

fig.show()

### Logistic Regression vs Decision Rules

In [158]:
# Logistic Regression
X_lr, Y_lr = synthetic_dataset_lr(n_samples=500, noise_scale=0.7, random_state=1505)
X_lr_lr_train, X_lr_lr_test, Y_lr_lr_train, Y_lr_lr_test, model_lr_lr = train_ml_model(model_type='logistic', X=X_lr, y=Y_lr, test_size=0.2, random_state=1505)
X_lr_dr_train, X_lr_dr_test, Y_lr_dr_train, Y_lr_dr_test, model_lr_dr = train_ml_model(model_type='decision_rules', X=X_lr, y=Y_lr, test_size=0.2, random_state=1505)

# Accuracy
print(f"Logistic Regression Accuracy: {accuracy_score(Y_lr_lr_test, model_lr_lr.predict(X_lr_lr_test))}")
print(f"Decision Rules Accuracy: {accuracy_score(Y_lr_dr_test, model_lr_dr.predict(X_lr_dr_test))}")


Logistic Regression Accuracy: 0.95
Decision Rules Accuracy: 0.25


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mlxtend/classifier/oner.py:89: UserWarning:

Feature array likely contains at least one non-categorical column. Column 0 appears to have a unique value in every row.



In [159]:
# Decision Rules
X_dr, Y_dr = synthetic_dataset_dr(n_samples=500, noise_scale=0.7, random_state=1505)
X_dr_lr_train, X_dr_lr_test, Y_dr_lr_train, Y_dr_lr_test, model_dr_lr = train_ml_model(model_type='logistic', X=X_dr, y=Y_dr, test_size=0.2, random_state=1505)
X_dr_dr_train, X_dr_dr_test, Y_dr_dr_train, Y_dr_dr_test, model_dr_dr = train_ml_model(model_type='decision_rules', X=X_dr, y=Y_dr, test_size=0.2, random_state=1505)

# Accuracy
print(f"Logistic Regression Accuracy: {accuracy_score(Y_dr_lr_test, model_dr_lr.predict(X_dr_lr_test))}")
print(f"Decision Rules Accuracy: {accuracy_score(Y_dr_dr_test, model_dr_dr.predict(X_dr_dr_test))}")


Logistic Regression Accuracy: 0.42
Decision Rules Accuracy: 0.25


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mlxtend/classifier/oner.py:89: UserWarning:

Feature array likely contains at least one non-categorical column. Column 0 appears to have a unique value in every row.



### SVM vs K Nearest Neighbors

In [126]:
# SVM Dataset
X_svm, Y_svm = synthetic_dataset_svm(n_samples=500, noise_scale=0.7, random_state=1505)
X_svm_svm_train, X_svm_svm_test, Y_svm_svm_train, Y_svm_svm_test, model_svm_svm = train_ml_model(model_type='svm', X=X_svm, y=Y_svm, test_size=0.2, random_state=1505)
X_svm_knn_train, X_svm_knn_test, Y_svm_knn_train, Y_svm_knn_test, model_svm_knn = train_ml_model(model_type='nearest_neighbors', X=X_svm, y=Y_svm, test_size=0.2, random_state=1505)

# Accuracy
print(f"SVM Accuracy: {accuracy_score(Y_svm_svm_test, model_svm_svm.predict(X_svm_svm_test))}")
print(f"KNN Accuracy: {accuracy_score(Y_svm_knn_test, model_svm_knn.predict(X_svm_knn_test))}")


SVM Accuracy: 0.96
KNN Accuracy: 0.95


In [ ]:
# KNN Dataset
X_knn, Y_knn = synthetic_dataset_knn(n_samples=500, noise_scale=0.7, random_state=1505)
X_knn_svm_train, X_knn_svm_test, Y_knn_svm_train, Y_knn_svm_test, model_knn_svm = train_ml_model(model_type='svm', X=X_knn, y=Y_knn, test_size=0.2, random_state=1505)
X_knn_knn_train, X_knn_knn_test, Y_knn_knn_train, Y_knn_knn_test, model_knn_knn = train_ml_model(model_type='nearest_neighbors', X=X_knn, y=Y_knn, test_size=0.2, random_state=1505)

# Accuracy
print(f"SVM Accuracy: {accuracy_score(Y_knn_svm_test, model_knn_svm.predict(X_knn_svm_test))}")
print(f"KNN Accuracy: {accuracy_score(Y_knn_knn_test, model_knn_knn.predict(X_knn_knn_test))}")


SVM Accuracy: 0.95
KNN Accuracy: 0.95
